# Model Explorations

In [1]:
from ema_workbench import ema_logging, MultiprocessingEvaluator
from dike_model_function import DikeNetwork
from ema_workbench import Model, RealParameter, ScalarOutcome, CategoricalParameter, IntegerParameter, BooleanParameter
# from ema_workbench.em_framework.evaluators import MC
ema_logging.log_to_stderr(ema_logging.INFO)

function = DikeNetwork()
# workbench model:
dike_model = Model('dikesnet', function=function)

# Uncertainties and Levers:
# Specify uncertainties range:
Real_uncert = {'Bmax': [30, 350], 'pfail': [0, 1]}  # m and [.]
# breach growth rate [m/day]
cat_uncert_loc = {'Brate': (1., 1.5, 10)}

cat_uncert = {'discount rate {}'.format(n): (1.5, 2.5, 3.5, 4.5)
                for n in function.planning_steps}

Int_uncert = {'A.0_ID flood wave shape': [0, 132]}
# Range of dike heightening:
dike_lev = {'DikeIncrease': [0, 10]}    # dm

# Series of five Room for the River projects:
rfr_lev = ['{}_RfR'.format(project_id) for project_id in range(0, 5)]

# Time of warning: 0, 1, 2, 3, 4 days ahead from the flood
EWS_lev = {'EWS_DaysToThreat': [0, 4]}  # days

uncertainties = []
levers = []

for uncert_name in cat_uncert.keys():
    categories = cat_uncert[uncert_name]
    uncertainties.append(CategoricalParameter(uncert_name, categories))

for uncert_name in Int_uncert.keys():
    uncertainties.append(IntegerParameter(uncert_name, 
                                            Int_uncert[uncert_name][0],
                                            Int_uncert[uncert_name][1]))    

# RfR levers can be either 0 (not implemented) or 1 (implemented)
for lev_name in rfr_lev:
    for n in function.planning_steps:
        lev_name_ = '{} {}'.format(lev_name, n)
        levers.append(IntegerParameter(lev_name_, 0, 1))

# Early Warning System lever
for lev_name in EWS_lev.keys():
    levers.append(IntegerParameter(lev_name, EWS_lev[lev_name][0],
                                    EWS_lev[lev_name][1]))

for dike in function.dikelist:
    # uncertainties in the form: locationName_uncertaintyName
    for uncert_name in Real_uncert.keys():
        name = "{}_{}".format(dike, uncert_name)
        lower, upper = Real_uncert[uncert_name]
        uncertainties.append(RealParameter(name, lower, upper))

    for uncert_name in cat_uncert_loc.keys():
        name = "{}_{}".format(dike, uncert_name)
        categories = cat_uncert_loc[uncert_name]
        uncertainties.append(CategoricalParameter(name, categories))

    # location-related levers in the form: locationName_leversName
    for lev_name in dike_lev.keys():
        for n in function.planning_steps:
            name = "{}_{} {}".format(dike, lev_name, n)
            levers.append(IntegerParameter(name, dike_lev[lev_name][0],
                                        dike_lev[lev_name][1]))

# load uncertainties and levers in dike_model:
dike_model.uncertainties = uncertainties
dike_model.levers = levers

variable_names = []
variable_names_ = []

for n in function.planning_steps:
    
    variable_names.extend(
        ['{}_{} {}'.format(dike, e, n) for e in [
            'Expected Annual Damage', 'Dike Investment Costs'] for dike in function.dikelist])

    variable_names_.extend(
        ['{}_{} {}'.format(dike, e, n) for e in [
            'Expected Number of Deaths'] for dike in function.dikelist])

    variable_names.extend(['RfR Total Costs {}'.format(n)])
    variable_names.extend(['Expected Evacuation Costs {}'.format(n)])

dike_model.outcomes = [ScalarOutcome('A.1_Expected Annual Damage 0'),
                            ScalarOutcome('A.1_Expected Number of Deaths 0')]

In [2]:
from ema_workbench import ema_logging, MultiprocessingEvaluator

ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=10,               #500
                                            policies=4,
                                            uncertainty_sampling='mc')

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 10 scenarios * 4 policies * 1 model(s) = 40 experiments
[MainProcess/INFO] 4 cases completed
[MainProcess/INFO] 8 cases completed
[MainProcess/INFO] 12 cases completed
[MainProcess/INFO] 16 cases completed
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 24 cases completed
[MainProcess/INFO] 28 cases completed
[MainProcess/INFO] 32 cases completed
[MainProcess/INFO] 36 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


##### @ Martajin, for unknown reasons, I'm not able to run the cell above(maybe it just runs super slow). I thus created another notebook and I used the way in the example ("ProblemFormulation.ipynb) to import model and perform experiments. It runs faster than this one! 

@Hongxuan, i f*'ed up. Sorry. Forgot to push my working results. But as you can see the problem was in the outcome. No real outcomes used in the model were defined. It was just a test run, with 10 scenarios to test if this works. This should also work for you. ALthough it still is slow. 

In [ ]:
experiments, outcomes = results

In [ ]:
import functools

def robustness(direction, threshold, data):
    if direction == SMALLER:
        return np.sum(data<=threshold)/data.shape[0]
    else:
        return np.sum(data>=threshold)/data.shape[0]

SMALLER = 'SMALLER'

Expected_Number_of_Deaths = functools.partial(robustness, SMALLER, 0.000001) #not ok
Expected_Annual_Damage = functools.partial(robustness, SMALLER, 1000) #THOSE NUMBERS NEED TO BE SPECIFIED AGAINS
Total_Investment_Costs = functools.partial(robustness, SMALLER, 150000000)#THOSE NUMBERS NEED TO BE SPECIFIED AGAINS

from problem_formulation import get_model_for_problem_formulation
from ema_workbench.em_framework.evaluators import MC

model = get_model_for_problem_formulation(0)
from ema_workbench import ema_logging, MultiprocessingEvaluator

ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(model) as evaluator:
    results = evaluator.perform_experiments(scenarios=200,               #500
                                            policies=4,
                                            uncertainty_sampling=MC)